In [1]:
# sentiment analysis with pyspark 

In [2]:
# Sentiment Analysis with Pyspark
import warnings
warnings.filterwarnings('ignore')

#%matplotlib inline
import pandas as pd

In [3]:

from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [4]:
from pyspark.sql import SQLContext
from pyspark import SparkContext

sqlContext = SQLContext(sc)

In [5]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='false', inferschema='true').load('/FileStore/tables/pyspark_learning/test/training_1600000_processed_noemoticon-efba6.csv')

In [6]:
pd.DataFrame(df.take(5), columns = df.columns).transpose()

,0,1,2,3,4
_c0,0,0,0,0,0
_c1,1467810369,1467810672,1467810917,1467811184,1467811193
_c2,Mon Apr 06 22:19:45 PDT 2009,Mon Apr 06 22:19:49 PDT 2009,Mon Apr 06 22:19:53 PDT 2009,Mon Apr 06 22:19:57 PDT 2009,Mon Apr 06 22:19:57 PDT 2009
_c3,NO_QUERY,NO_QUERY,NO_QUERY,NO_QUERY,NO_QUERY
_c4,_TheSpecialOne_,scotthamilton,mattycus,ElleCTF,Karoli
_c5,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",is upset that he can't update his Facebook by ...,@Kenichan I dived many times for the ball. Man...,my whole body feels itchy and like its on fire,"@nationwideclass no, it's not behaving at all...."


In [7]:
df.show(5)

+---+----------+--------------------+--------+---------------+--------------------+
_c0| _c1| _c2| _c3| _c4| _c5|
+---+----------+--------------------+--------+---------------+--------------------+
 0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
 0|1467810672|Mon Apr 06 22:19:...|NO_QUERY| scotthamilton|is upset that he ...|
 0|1467810917|Mon Apr 06 22:19:...|NO_QUERY| mattycus|@Kenichan I dived...|
 0|1467811184|Mon Apr 06 22:19:...|NO_QUERY| ElleCTF|my whole body fee...|
 0|1467811193|Mon Apr 06 22:19:...|NO_QUERY| Karoli|@nationwideclass ...|
+---+----------+--------------------+--------+---------------+--------------------+
only showing top 5 rows

In [8]:
df = df.dropna()

In [9]:
df.columns

Out[8]: ['_c0', '_c1', '_c2', '_c3', '_c4', '_c5']

In [10]:
df.count()

Out[9]: 1600000

In [11]:
(train_set, val_set, test_set) = df.randomSplit([0.98, 0.01, 0.01], seed = 2000)

In [12]:
# HashingTF + IDF + Logistic Regression

In [13]:
tokenizer = Tokenizer(inputCol='_c5', outputCol="words")
hashtf = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx = StringIndexer(inputCol = "_c0", outputCol = "label")
pipeline = Pipeline(stages=[tokenizer, hashtf, idf, label_stringIdx])

In [14]:
train_set.show()

+---+----------+--------------------+--------+---------------+--------------------+
_c0| _c1| _c2| _c3| _c4| _c5|
+---+----------+--------------------+--------+---------------+--------------------+
 0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
 0|1467810672|Mon Apr 06 22:19:...|NO_QUERY| scotthamilton|is upset that he ...|
 0|1467810917|Mon Apr 06 22:19:...|NO_QUERY| mattycus|@Kenichan I dived...|
 0|1467811184|Mon Apr 06 22:19:...|NO_QUERY| ElleCTF|my whole body fee...|
 0|1467811193|Mon Apr 06 22:19:...|NO_QUERY| Karoli|@nationwideclass ...|
 0|1467811372|Mon Apr 06 22:20:...|NO_QUERY| joy_wolf|@Kwesidei not the...|
 0|1467811592|Mon Apr 06 22:20:...|NO_QUERY| mybirch| Need a hug |
 0|1467811594|Mon Apr 06 22:20:...|NO_QUERY| coZZ|@LOLTrish hey lo...|
 0|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nope t...|
 0|1467812025|Mon Apr 06 22:20:...|NO_QUERY| mimismo|@twittera que me ...|
 0|1467812416|Mon Apr 06 22:20:...|NO_QUERY| erinx3leannexo|spring break in p...|
 0|1467812579|Mon Apr 06 22:20:...|NO_QUERY| pardonlauren|I just re-pierced...|
 0|1467812723|Mon Apr 06 22:20:...|NO_QUERY| TLeC|@caregiving I cou...|
 0|1467812771|Mon Apr 06 22:20:...|NO_QUERY|robrobbierobert|@octolinz16 It it...|
 0|1467812784|Mon Apr 06 22:20:...|NO_QUERY| bayofwolves|@smarrison i woul...|
 0|1467812799|Mon Apr 06 22:20:...|NO_QUERY| HairByJess|@iamjazzyfizzle I...|
 0|1467812964|Mon Apr 06 22:20:...|NO_QUERY| lovesongwriter|Hollis' death sce...|
 0|1467813137|Mon Apr 06 22:20:...|NO_QUERY| armotley|about to file taxes |
 0|1467813579|Mon Apr 06 22:20:...|NO_QUERY| starkissed|@LettyA ahh ive a...|
 0|1467813782|Mon Apr 06 22:20:...|NO_QUERY| gi_gi_bee|@FakerPattyPattz ...|
+---+----------+--------------------+--------+---------------+--------------------+
only showing top 20 rows

In [15]:
pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
val_df = pipelineFit.transform(val_set)
train_df.show(5)

+---+----------+--------------------+--------+---------------+--------------------+--------------------+--------------------+--------------------+-----+
_c0| _c1| _c2| _c3| _c4| _c5| words| tf| features|label|
+---+----------+--------------------+--------+---------------+--------------------+--------------------+--------------------+--------------------+-----+
 0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|[@switchfoot, htt...|(65536,[8436,8847...|(65536,[8436,8847...| 1.0|
 0|1467810672|Mon Apr 06 22:19:...|NO_QUERY| scotthamilton|is upset that he ...|[is, upset, that,...|(65536,[1444,2071...|(65536,[1444,2071...| 1.0|
 0|1467810917|Mon Apr 06 22:19:...|NO_QUERY| mattycus|@Kenichan I dived...|[@kenichan, i, di...|(65536,[2888,3924...|(65536,[2888,3924...| 1.0|
 0|1467811184|Mon Apr 06 22:19:...|NO_QUERY| ElleCTF|my whole body fee...|[my, whole, body,...|(65536,[158,11650...|(65536,[158,11650...| 1.0|
 0|1467811193|Mon Apr 06 22:19:...|NO_QUERY| Karoli|@nationwideclass ...|[@nationwideclass...|(65536,[1968,3434...|(65536,[1968,3434...| 1.0|
+---+----------+--------------------+--------+---------------+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 5 rows

In [16]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=100)
lrModel = lr.fit(train_df)
predictions = lrModel.transform(val_df)

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

Out[16]: 0.8563625270558153

In [18]:
evaluator.getMetricName()

Out[17]: 'areaUnderROC'

In [19]:
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(val_set.count())
accuracy

Out[18]: 0.7831772175771673

In [20]:
# CountVectorizer + IDF + Logistic Regression

In [21]:
%%time
from pyspark.ml.feature import CountVectorizer

tokenizer = Tokenizer(inputCol="_c5", outputCol="words")
cv = CountVectorizer(vocabSize=2**16, inputCol="words", outputCol='cv')
idf = IDF(inputCol='cv', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx = StringIndexer(inputCol = "_c0", outputCol = "label")
lr = LogisticRegression(maxIter=100)
pipeline = Pipeline(stages=[tokenizer, cv, idf, label_stringIdx, lr])

pipelineFit = pipeline.fit(train_set)
predictions = pipelineFit.transform(val_set)
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(val_set.count())
roc_auc = evaluator.evaluate(predictions)

print("Accuracy Score: {0:.4f}".format(accuracy))
print("ROC-AUC: {0:.4f}".format(roc_auc))

Accuracy Score: 0.7972
ROC-AUC: 0.8707
CPU times: user 464 ms, sys: 169 ms, total: 633 ms
Wall time: 1min 48s